In [1]:
# import data

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os
import pandas as pd

# import numpy and relevant sklearn modules
import numpy as np

# Datetime
from datetime import datetime

#Keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import tensorflow 
from tensorflow import keras
from tensorflow.keras import layers

#Import LSTM
from keras.layers import LSTM
#Import Dropout
from tensorflow.keras.layers import Dropout

#Import StandardScaler
from sklearn.preprocessing import StandardScaler

In [2]:
#Read in EV/EBITDA historical ratio over five years from 2017-2022
df = pd.read_csv("/content/drive/MyDrive/MachineLearning/EBITDA Project/AAPL_quarterly_valuation_measures.csv")
df


,Date,EnterprisesValueEBITDARatio
0,6/30/22,85.334
1,3/31/22,86.063
2,12/31/21,66.404
3,9/30/21,88.613
4,6/30/21,83.236
5,3/31/21,65.870
6,12/31/20,61.106
7,9/30/20,108.887
8,6/30/20,95.155
9,3/31/20,65.996


In [3]:
# Let's check our columns
df.columns

Index(['Date', 'EnterprisesValueEBITDARatio'], dtype='object')

In [4]:
# Making sure the Date column is in Datetime might make it more readable.
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,EnterprisesValueEBITDARatio
0,2022-06-30,85.334
1,2022-03-31,86.063
2,2021-12-31,66.404
3,2021-09-30,88.613
4,2021-06-30,83.236
5,2021-03-31,65.870
6,2020-12-31,61.106
7,2020-09-30,108.887
8,2020-06-30,95.155
9,2020-03-31,65.996


In [5]:
# We need to make sure these dates are business quarter end dates.
df_output = df.loc[df.Date.isin(df.Date + pd.offsets.BQuarterEnd(1))]
df_output

,Date,EnterprisesValueEBITDARatio
0,2022-06-30,85.334
1,2022-03-31,86.063
2,2021-12-31,66.404
3,2021-09-30,88.613
4,2021-06-30,83.236
5,2021-03-31,65.870
6,2020-12-31,61.106
7,2020-09-30,108.887
8,2020-06-30,95.155
9,2020-03-31,65.996


In [6]:
# We need to insert one date that somehow got missed.
df_output1 = df_output.append({'Date':'2017-06-30', 'EnterprisesValueEBITDARatio':'54.846'}, ignore_index=True)
df_output1

,Date,EnterprisesValueEBITDARatio
0,2022-06-30 00:00:00,85.334
1,2022-03-31 00:00:00,86.063
2,2021-12-31 00:00:00,66.404
3,2021-09-30 00:00:00,88.613
4,2021-06-30 00:00:00,83.236
5,2021-03-31 00:00:00,65.87
6,2020-12-31 00:00:00,61.106
7,2020-09-30 00:00:00,108.887
8,2020-06-30 00:00:00,95.155
9,2020-03-31 00:00:00,65.996


In [7]:
# Run Datetime again to integrate the date correctly
df_output1['Date'] = pd.to_datetime(df_output1['Date'])
df_output1

,Date,EnterprisesValueEBITDARatio
0,2022-06-30,85.334
1,2022-03-31,86.063
2,2021-12-31,66.404
3,2021-09-30,88.613
4,2021-06-30,83.236
5,2021-03-31,65.87
6,2020-12-31,61.106
7,2020-09-30,108.887
8,2020-06-30,95.155
9,2020-03-31,65.996


In [8]:
#Reset our index
df_output1.reset_index(drop='True')

,Date,EnterprisesValueEBITDARatio
0,2022-06-30,85.334
1,2022-03-31,86.063
2,2021-12-31,66.404
3,2021-09-30,88.613
4,2021-06-30,83.236
5,2021-03-31,65.87
6,2020-12-31,61.106
7,2020-09-30,108.887
8,2020-06-30,95.155
9,2020-03-31,65.996


2. Now Find the Stock Value Information

In [9]:
# read in the Stock Price csv
df2 = pd.read_csv("/content/drive/MyDrive/MachineLearning/EBITDA Project/AAPL.csv")
df2.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-05-30,38.355000,38.607498,38.332500,38.417500,36.318096,80507600
1,2017-05-31,38.492500,38.542500,38.095001,38.189999,36.103031,97804800
2,2017-06-01,38.292500,38.332500,38.055000,38.294998,36.202282,65616400
3,2017-06-02,38.395000,38.862499,38.222500,38.862499,36.738773,111082800
4,2017-06-05,38.584999,38.612499,38.365002,38.482498,36.379547,101326800


In [10]:
# Winnow down the columns
df2.drop(df2.iloc[:, 1:4], inplace=True, axis=1)
df2.head()

,Date,Close,Adj Close,Volume
0,2017-05-30,38.417500,36.318096,80507600
1,2017-05-31,38.189999,36.103031,97804800
2,2017-06-01,38.294998,36.202282,65616400
3,2017-06-02,38.862499,36.738773,111082800
4,2017-06-05,38.482498,36.379547,101326800


In [11]:
# Winnow them some more
df2.drop(df2.iloc[:, 2:], inplace=True, axis=1)
df2

,Date,Close
0,2017-05-30,38.417500
1,2017-05-31,38.189999
2,2017-06-01,38.294998
3,2017-06-02,38.862499
4,2017-06-05,38.482498
...,...,...
1305,2022-08-04,165.809998
1306,2022-08-05,165.350006
1307,2022-08-08,164.869995
1308,2022-08-09,164.919998


In [12]:
# Convert to datetime
df2['Date'] = pd.to_datetime(df2['Date']) # -- if column is not datetime yet
df2

,Date,Close
0,2017-05-30,38.417500
1,2017-05-31,38.189999
2,2017-06-01,38.294998
3,2017-06-02,38.862499
4,2017-06-05,38.482498
...,...,...
1305,2022-08-04,165.809998
1306,2022-08-05,165.350006
1307,2022-08-08,164.869995
1308,2022-08-09,164.919998


In [13]:
# Winnow the dates that are only business quarter end dates to match the EV/EBITDA csv
df2_output = df2.loc[df2.Date.isin(df2.Date + pd.offsets.BQuarterEnd(1))]
df2_output

,Date,Close
23,2017-06-30,36.005001
86,2017-09-29,38.529999
149,2017-12-29,42.307499
274,2018-06-29,46.277500
337,2018-09-28,56.435001
400,2018-12-31,39.435001
461,2019-03-29,47.487499
524,2019-06-28,49.480000
588,2019-09-30,55.992500
652,2019-12-31,73.412498


In [14]:
# We must reset the index because a lot of dates are now gone.
df2_output.reset_index(drop=True)

,Date,Close
0,2017-06-30,36.005001
1,2017-09-29,38.529999
2,2017-12-29,42.307499
3,2018-06-29,46.277500
4,2018-09-28,56.435001
5,2018-12-31,39.435001
6,2019-03-29,47.487499
7,2019-06-28,49.480000
8,2019-09-30,55.992500
9,2019-12-31,73.412498


In [15]:
# Again, like the first csv, there seems to be one date the Yahoo Finance! csv didn't have
df2_output.loc[2.5] = ['2018-03-30', 45.513]
df2_output = df2_output.sort_index().reset_index(drop=True)
df2_output

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,Date,Close
0,2018-03-30,45.513000
1,2017-06-30 00:00:00,36.005001
2,2017-09-29 00:00:00,38.529999
3,2017-12-29 00:00:00,42.307499
4,2018-06-29 00:00:00,46.277500
5,2018-09-28 00:00:00,56.435001
6,2018-12-31 00:00:00,39.435001
7,2019-03-29 00:00:00,47.487499
8,2019-06-28 00:00:00,49.480000
9,2019-09-30 00:00:00,55.992500


In [16]:
# Make sure that missing date is in datetime
df2_output['Date'] = pd.to_datetime(df2_output['Date'])
df2_output

,Date,Close
0,2018-03-30,45.513000
1,2017-06-30,36.005001
2,2017-09-29,38.529999
3,2017-12-29,42.307499
4,2018-06-29,46.277500
5,2018-09-28,56.435001
6,2018-12-31,39.435001
7,2019-03-29,47.487499
8,2019-06-28,49.480000
9,2019-09-30,55.992500


In [17]:
# let's order by the date.
df2_output.sort_values(by='Date')

,Date,Close
1,2017-06-30,36.005001
2,2017-09-29,38.529999
3,2017-12-29,42.307499
0,2018-03-30,45.513000
4,2018-06-29,46.277500
5,2018-09-28,56.435001
6,2018-12-31,39.435001
7,2019-03-29,47.487499
8,2019-06-28,49.480000
9,2019-09-30,55.992500


In [18]:
# What are our columns?
df2_output.columns

Index(['Date', 'Close'], dtype='object')

In [19]:
# We're done! 
data = pd.merge(df_output1, df2_output)
data


,Date,EnterprisesValueEBITDARatio,Close
0,2022-06-30,85.334,136.720001
1,2022-03-31,86.063,174.610001
2,2021-12-31,66.404,177.570007
3,2021-09-30,88.613,141.500000
4,2021-06-30,83.236,136.960007
5,2021-03-31,65.87,122.150002
6,2020-12-31,61.106,132.690002
7,2020-09-30,108.887,115.809998
8,2020-06-30,95.155,91.199997
9,2020-03-31,65.996,63.572498
